In [5]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

# Engaging GPU
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
torch.cuda.is_available()

In [6]:
device

device(type='cuda')

# Data Preprocessing

In [7]:
# Define relevant variables for the ML task
batch_size = 64
num_classes = 7
learning_rate = 0.001
num_epochs = 2

In [8]:
train_trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

valid_trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [9]:
from torchvision.datasets import ImageFolder
train_folder = ImageFolder("D:\\FALL 2022\\MS_Thesis\\MASTER'S THESIS\\EdmCrack600\\Data_Structure(Annotated)", transform=train_trans , )
test_folder = ImageFolder("D:\\FALL 2022\\MS_Thesis\\MASTER'S THESIS\\EdmCrack600\\Data_Structure(Annotated)", transform=valid_trans , )

In [10]:
len(train_folder.classes)

7

In [11]:
train_folder.classes

['ALLIGATOR ( Fatigue plus Longitudinal Cracks on Wheel Paths',
 'BLEEDING_',
 'DELAMINATION',
 'LONGITUDINAL LANE JOINT CRACKING_',
 'LONGITUDINAL OUTSIDE OF THE WHEEL PATHS_',
 'Raveling',
 'Transverse Crack']

In [12]:
classes_name = train_folder.classes
classes_name

['ALLIGATOR ( Fatigue plus Longitudinal Cracks on Wheel Paths',
 'BLEEDING_',
 'DELAMINATION',
 'LONGITUDINAL LANE JOINT CRACKING_',
 'LONGITUDINAL OUTSIDE OF THE WHEEL PATHS_',
 'Raveling',
 'Transverse Crack']

In [13]:
# Splitting data on batches
train_loader = torch.utils.data.DataLoader(train_folder, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_folder, shuffle=False, batch_size=batch_size)

In [14]:
# Let's take a look at the first batch
data, labels = next(iter(train_loader))
# data, labels = data.cpu(), labels.cpu() ## Solution: https://stackoverflow.com/questions/59013109/runtimeerror-input-type-torch-floattensor-and-weight-type-torch-cuda-floatte

In [15]:
data.size

<function Tensor.size>

data.shape

In [16]:
type(data)

torch.Tensor

1. The input of a Pytorch Neural Network is of type [BATCH_SIZE] * [CHANNEL_NUMBER] * [HEIGHT] * [WIDTH]

2. 1×3×32×32 meaning that you have 1 image with 3 channels (RGB) with height 32 and width 32

3. . The formular of convolution is ((W - F + 2P)/ S )+1 and ((H - F + 2P)/ S )+1

4. W =  WIDTH, F  = FILTER_SIZE, P = PADDIND, S = STRIDE

with our input 1×3×32×32 after applying conv1 W will be 28 and H will be 28 and also applying (2,2) pooling halves the WIDTH and HEIGTH and We have 6 feature maps. So after the first con2d and pooling we end up with an image of dimension
1 * 6 * 14 * 14. Similaly for the second conv2d and pooling we will end up with an image of dimension 1 * 16 * 5 * 5 . Finally since we need a column vector for the first fc layer we should unroll our vector which is 16×5×5 = 400

https://discuss.pytorch.org/t/input-size-of-fc-layer-in-tutorial/14644


## Here, the given relationship between the kernel size and padding is that padding = (kernel size - 1) / 2. Max pooling with kernel_size = stride = 2 will decrease the width/height by a factor of 2 (rounded down if input shape is not even).



## CALL VGG 19 from scratch

In [64]:
class VGG19(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG19, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())                        ##  64 X 224 X 224
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2)) ##  64 X 112 X 112
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())                       ##  128 X 112 X 112
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))             ##  128 X 56 X 56
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())                           ##  256 X 56 X 56
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())                              ##  256 X 56 X 56
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())                                      ##  256 X 56 X 56
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))        ##  256 X 28 X 28
        self.layer9 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())                           ##  512 X 28 X 28
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())                     ##  512 X 28 X 28
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())                   ##  512 X 28 X 28
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))             ##  512 X 14 X 14
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())                      ##  512 X 14 X 14
        self.layer14 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())                     ##  512 X 14 X 14
        self.layer15 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())                       ##  512 X 14 X 14
        self.layer16 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))      ##  512 X 7 X 7
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),  # (height & width * (filter = 512))
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        print(out.size())
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [65]:
model = VGG19() 
model = model.to(device=device)

In [66]:
model.classifier[6] = nn.Linear(4096, 7).to(device) # original model has outputs for 1000 classes. 
# But there are only 75 classes so we have to change output layer

AttributeError: 'VGG19' object has no attribute 'classifier'

In [ ]:
model

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))

In [ ]:
# Freezing all layers except last 15
for param in list(model.parameters())[:-15]:
    param.requires_grad = False

In [ ]:
# Defining model optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))

## Hyperameter

In [44]:
# Define relevant variables for the ML task
batch_size = batch_size 
num_classes = len(train_folder.classes)
learning_rate = learning_rate
num_epochs = num_epochs

model = VGG19(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(train_loader)

## Training

In [45]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 4.00 GiB total capacity; 2.86 GiB already allocated; 0 bytes free; 2.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF